In [18]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
from tensorflow import keras, convert_to_tensor, reshape
from Lenet import LeNet
import pandas as pd

In [2]:
label_dict = pd.read_csv('label_dict.csv')
label_dict_inv = pd.read_csv('label_dict_inv.csv')

In [46]:
def preprocess_keras(img):
#     cv2.imshow("input",img)
#     cv2.waitKey(0)
#     print(f"Img shape before: {img.shape}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_inv = cv2.bitwise_not(gray)
    _, binary = cv2.threshold(gray_inv, 35, 255, cv2.THRESH_BINARY)
    # get the indices of the black pixels
    y, x = np.nonzero(binary)
    # get the maximum and minimum x and y coordinates
    max_x = np.max(x)
    min_x = np.min(x)
    max_y = np.max(y)
    min_y = np.min(y)
    nonzero_indices = np.nonzero(binary)
    zero_indices = np.where(binary == 0)
    darkened_gray = gray.astype('float32')
    darkened_gray[zero_indices] = 229+0.1*darkened_gray[zero_indices]
    darkened_gray[nonzero_indices] *= 0.1
    darkened_gray = darkened_gray.astype('uint8')
    cropped = darkened_gray[min_y:max_y, min_x:max_x]
    # _, thresh_img = cv2.threshold(gray_roi, 155, 255, cv2.THRESH_BINARY)
    inverted_img = cv2.bitwise_not(cropped)
    aspect_ratio = inverted_img.shape[1] / inverted_img.shape[0]
    # resize the image while keeping the aspect ratio
    new_height = 20
    new_width = int(new_height * aspect_ratio)
    if new_width > new_height:
        new_width = 20
        aspect_ratio = inverted_img.shape[0] / inverted_img.shape[1]
        new_height = int(new_width * aspect_ratio)
    resized_img = cv2.resize(inverted_img, (new_width, new_height))
    # pad the resized image with black pixels to make it 28x28
    pad_width = (
        ((28 - new_height) // 2, (28 - new_height + 1) // 2),  # no padding on top and bottom
        ((28 - new_width) // 2, (28 - new_width + 1) // 2))  # pad equally on both sides to make the width 28
    padded_img = np.pad(resized_img, pad_width, mode='constant', constant_values=0)
    # img_brighter = cv2.add(inverted_img, 50)
    normalized_img = (cv2.resize(padded_img, (28, 28)))
#     cv2.imshow("preprocessed image",normalized_img)
#     cv2.waitKey(0)
    img_tensor = convert_to_tensor(normalized_img)
    img_tensor /= 255
    img_tensor = reshape(img_tensor, [1, 28, 28, 1])
    img_tensor = np.array(img_tensor)
#     print(f"Img shape after: {img_tensor.shape}")
#     print(f"Tensor type: {type(img_tensor)}")
#     print(f"Tensor: {img_tensor}")
#     cv2.destroyAllWindows()
#     normalized_tensor = 2 * (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min()) - 1
    return img_tensor


def keras_preprocessing(img):
    RGB = 1
    img = img.reshape(img.shape[0], img.shape[1], img.shape[2], RGB)
    img = img/255
    return img

In [30]:
def extract_characters(img_name, model):
    img = cv2.imread(img_name)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 5)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_SIMPLE)
    fields_points_sorted = []
    # Loop through the contours
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        threshold_value = 200
        black_found = False
        character = img[max((y + 3), 0):y + h - 3, max((x + 3), 0):x + w - 3]
        # Iterate over all pixels and check if their intensity values are above the threshold
        for row in range(character.shape[0]):
            for col in range(character.shape[1]):
                pixel_intensity = character[row, col]
                if (pixel_intensity < threshold_value).any():
                    black_found = True
                    break
            if black_found:
                break
        # Get the bounding rectangle of the contour
        if 30 < w < 100 and 40 < h < 100 and black_found:
            fields_points_sorted.append([max((y + 3), 0), y + h - 3, max((x + 3), 0), x + w - 3])
            # cv2.imwrite("C:/Users/30698/Desktop/KULeuven/Capita Selecta/test.jpeg", character)
            # show_one_prediction(character,model)
            #cv2.rectangle(img, (x + 3, y + 3), (x + w - 3, y + h - 3), (0, 255, 0), 2)
    fields_points_sorted.sort(key=lambda r: r[2])
    characters_sorted = []
    for field_points in fields_points_sorted:
        characters_sorted.append(img[field_points[0]:field_points[1], field_points[2]:field_points[3]])

    # Display the image with bounding boxes
#     cv2.imshow('image', characters_sorted[4])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return characters_sorted


def show_one_prediction(img, model, i=0):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_inv = cv2.bitwise_not(gray)
    _, binary = cv2.threshold(gray_inv, 25, 255, cv2.THRESH_BINARY)
    # get the indices of the black pixels
    y, x = np.nonzero(binary)

    # get the maximum and minimum x and y coordinates
    max_x = np.max(x)
    min_x = np.min(x)
    max_y = np.max(y)
    min_y = np.min(y)
    nonzero_indices = np.nonzero(binary)
    darkened_gray = gray.astype('float32')
    darkened_gray[nonzero_indices] *= 0.3
    darkened_gray[not nonzero_indices] *= 1.1
    darkened_gray = darkened_gray.astype('uint8')
    cropped = darkened_gray[min_y:max_y, min_x:max_x]
    cv2.imshow("bin", cropped)
    cv2.waitKey(0)
    # _, thresh_img = cv2.threshold(gray_roi, 155, 255, cv2.THRESH_BINARY)
    inverted_img = cv2.bitwise_not(cropped)
    aspect_ratio = inverted_img.shape[1] / inverted_img.shape[0]

    # resize the image while keeping the aspect ratio
    new_height = 24
    new_width = int(new_height * aspect_ratio)
    if new_width > new_height:
        return
    resized_img = cv2.resize(inverted_img, (new_width, new_height))

    # pad the resized image with black pixels to make it 28x28
    pad_width = (
        ((28 - new_height) // 2, (28 - new_height + 1) // 2),  # no padding on top and bottom
        ((28 - new_width) // 2, (28 - new_width + 1) // 2))  # pad equally on both sides to make the width 28
    padded_img = np.pad(resized_img, pad_width, mode='constant', constant_values=0)
    # img_brighter = cv2.add(inverted_img, 50)
    normalized_img = (cv2.resize(padded_img, (28, 28)))
    # Turn off gradients to speed up this part
    img_tensor = torch.from_numpy(normalized_img).unsqueeze(0)
    normalized_tensor = 2 * (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min()) - 1
    # Define a transform to normalize the data
    # Apply the transform to the tensor
    print(normalized_tensor)
    with torch.no_grad():
        logps = model(normalized_tensor)

    # Output of the network are log-probabilities, need to take exponential for probabilities
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    # print(("pred:", probab.index(max(probab))))
    # print(("pred:", ))
    view_classify(img_tensor, ps)
    cv2.imshow("actual", img)
    cv2.waitKey(0)


def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6, 9), ncols=2)
    ax1.imshow(img.numpy().squeeze(), cmap='gray_r')
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)
    plt.tight_layout()
    plt.show()


In [5]:
people = [
    ['MICHELLE', 'RODRIGUEZ', '12/02/1950', 'RFMR12025033445', 'DE1202507171'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['ANGELA', 'FITZGERALD', '04/07/1956', 'RFAF04075656622', 'BE0407563203'],
    ['DAVID', 'LOPEZ', '17/01/1963', 'RFDL17016329220', 'BR1701638007'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['DEREK', 'CHURCH', '13/01/1972', 'RFDC13017297192', 'BL1301728110'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['EDWIN', 'ALLEN', '22/09/1977', 'RFEA22097775609', 'DE2209776709'],
    ['ANGEL', 'SCHWARTZ', '18/05/1982', 'RFAS18058263146', 'BR1805822377'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['MELISSA', 'MENDOZA', '11/06/1987', 'RFMM11068724346', 'BL1106877773'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343']

]


true_labels=[
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['MICHELLE', 'RODRIGUEZ', '12/02/1950', 'RFMR12025033445', 'DE1202507171'],
    ['DEREK', 'CHURCH', '13/01/1972', 'RFDC13017297192', 'BL1301728110'],
    ['DAVID', 'LOPEZ', '17/01/1963', 'RFDL17016329220', 'BR1701638007'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['EDWIN', 'ALLEN', '22/09/1977', 'RFEA22097775609', 'DE2209776709'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['MELISSA', 'MENDOZA', '11/06/1987', 'RFMM11068724346', 'BL1106877773'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['ANGEL', 'SCHWARTZ', '18/05/1982', 'RFAS18058263146', 'BR1805822377'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['ANGELA', 'FITZGERALD', '04/07/1956', 'RFAF04075656622', 'BE0407563203'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793']
]



In [91]:
def predict_date(model, characters_for_recognition, labels):
    i=0
    correct_fields=0
    for char in labels:
        if char != '/':
            processed_input = preprocess_keras(characters_for_recognition[i])
            argmax = str(np.argmax(model.predict(processed_input)))
            pred = str(label_dict_inv[argmax].get(0))
            print(f"Prediction: {pred}")
            # if int(probab.index(max(probab))) == int(char):
            if pred == char:
                correct_fields+=1
        i += 1
    return correct_fields

In [12]:
# model_digits = LeNet()
# model_digits.load_state_dict(torch.load('saved_models/digits_model_nn3'))
model_digits = keras.models.load_model('./alphanum_model')

print("Model successfully loaded")

# extract_characters('C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fields/0001_DATE.jpg')
# dir_path = 'C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fione

# Define the allowed image extensions
img_extensions = ["DATE.jpg", "DATE.jpeg", "DATE.png"]

Model successfully loaded


In [ ]:
for root, dirs, files in os.walk(fields_dir_path):
        # Iterate over all files in the current directory
        correct=0
        print(f"Found {len(files)} files in '{fields_dir_path}'")
        for file in files:
            # Check if the file has an image extension            
            if any(file.endswith(ext) for ext in img_extensions):
                print(f"Checking for {file}:")
                # Add the full path of the image to the list
                img_path = os.path.join(root, file)
                characters_of_date = extract_characters(img_path, model_digits)
                file_index = int(file[:4])
                print(file_index)
                correct += predict_date(model_digits, characters_of_date, true_labels[file_index-1][2])                
        print("Accuracy:", correct/(44*8))

Found 220 files in './Forms 24022023/fields'
Checking for 0017_DATE.jpg:
17
1/1 [==============================] - 0s 27ms/step
Prediction: 9
1/1 [==============================] - 0s 13ms/step
Prediction: 2
1/1 [==============================] - 0s 13ms/step
Prediction: 0
1/1 [==============================] - 0s 13ms/step
Prediction: q
1/1 [==============================] - 0s 14ms/step
Prediction: |
1/1 [==============================] - 0s 13ms/step
Prediction: Q
1/1 [==============================] - 0s 14ms/step
Prediction: A
1/1 [==============================] - 0s 14ms/step
Prediction: 4
Checking for 0011_DATE.jpg:
11
1/1 [==============================] - 0s 14ms/step
Prediction: 1
1/1 [==============================] - 0s 14ms/step
Prediction: g
1/1 [==============================] - 0s 14ms/step
Prediction: 0
1/1 [==============================] - 0s 13ms/step
Prediction: 1
1/1 [==============================] - 0s 13ms/step
Prediction: 1
1/1 [==============================

1/1 [==============================] - 0s 14ms/step
Prediction: 4
1/1 [==============================] - 0s 13ms/step
Prediction: g
1/1 [==============================] - 0s 15ms/step
Prediction: g
1/1 [==============================] - 0s 13ms/step
Prediction: S
Checking for 0023_DATE.jpg:
23
1/1 [==============================] - 0s 14ms/step
Prediction: l
1/1 [==============================] - 0s 13ms/step
Prediction: 7
1/1 [==============================] - 0s 14ms/step
Prediction: O
1/1 [==============================] - 0s 13ms/step
Prediction: Q
1/1 [==============================] - 0s 13ms/step
Prediction: L
1/1 [==============================] - 0s 13ms/step
Prediction: q
1/1 [==============================] - 0s 14ms/step
Prediction: q
1/1 [==============================] - 0s 13ms/step
Prediction: O
Checking for 0019_DATE.jpg:
19
1/1 [==============================] - 0s 13ms/step
Prediction: I
1/1 [==============================] - 0s 13ms/step
Prediction: l
1/1 [=========